#FEDERATED LEARNING

In [1]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_breast_cancer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler


In [2]:
data = load_breast_cancer()
X = data.data
y = data.target

print("Dataset shape:", X.shape)


Dataset shape: (569, 30)


In [3]:
NUM_CLIENTS = 3

X_clients = np.array_split(X, NUM_CLIENTS)
y_clients = np.array_split(y, NUM_CLIENTS)

print("Clients created:", NUM_CLIENTS)

Clients created: 3


In [4]:
def add_noise(weights, noise_level):
    noise = np.random.normal(0, noise_level, size=weights.shape)
    return weights + noise

In [5]:
def train_local_model(X, y):
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    model = LogisticRegression(max_iter=200)
    model.fit(X_scaled, y)

    return model.coef_[0]


In [6]:
def federated_average(weights_list):
    return np.mean(weights_list, axis=0)


In [7]:
def evaluate(global_weights, X, y):
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    model = LogisticRegression(max_iter=200)
    model.coef_ = np.array([global_weights])
    model.intercept_ = np.array([0.0])
    model.classes_ = np.array([0, 1])

    preds = model.predict(X_scaled)
    return accuracy_score(y, preds)


In [8]:
noise_levels = [0.0, 0.1, 0.5, 1.0]
runs = 3

results = {}

for noise in noise_levels:
    acc_list = []

    for _ in range(runs):
        local_weights = []

        # Each client trains locally
        for i in range(NUM_CLIENTS):
            w = train_local_model(X_clients[i], y_clients[i])
            w_noisy = add_noise(w, noise)
            local_weights.append(w_noisy)

        # Server aggregates
        global_weights = federated_average(local_weights)

        # Evaluate global model
        acc = evaluate(global_weights, X, y)
        acc_list.append(acc)

    results[noise] = np.mean(acc_list)


In [9]:
results_df = pd.DataFrame({
    "Noise Level": results.keys(),
    "Average Accuracy": results.values()
})

results_df


,Noise Level,Average Accuracy
0,0.0,0.978910
1,0.1,0.980082
2,0.5,0.960164
3,1.0,0.943761
